<a href="https://colab.research.google.com/github/mahault/bin_packing_mdp/blob/main/Bin_packing_mdp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Bin Packing Problem

Pack \( N \) items of different sizes into a finite number of bins or containers, each of a fixed given capacity, in a way that minimizes the number of bins used.

**Decision Variables:**
- \( x_{ij} \in \{0,1\} \): Denotes assigning item \( i \) to bin \( j \).
- \( w_i \in \mathbb{R} \): Size of each item.
- \( W_j \in \mathbb{R} \): Capacity of each bin.
- \( y_j \in \{0,1\} \): Denotes the event that bin \( j \) has been assigned at least one item.

**Constraints:**
- **Every item must be assigned to a single bin:**  
  \[
  \sum_{j} x_{ij} = 1, \forall j \in \{1, \ldots, \infty\}
  \]
- **Every nonempty bin gets turned on:**  
  \[
  y_j = \mathbb{I}\left[\sum_{i} x_{ij} > 0\right], \forall j \in \{1, \ldots, \infty\}
  \]
- **Sum of item sizes in each bin must not exceed bin capacity:**  
  \[
  \sum_{i} w_i x_{ij} \leq W_j, \forall j \in \{1, \ldots, \infty\}
  \]

**LP Formulation:**
\[
\text{min} \quad \sum_{j} y_j
\]

subject to:

- \[
  \sum_{j} x_{ij} = 1, \forall i \in \{1, \ldots, \infty\}
  \]
- \[  y_j = \mathbb{I}\left[\sum_{i} x_{ij} > 0\right], \forall j \in \{1, \ldots, \infty\}
  \]
- \[  \sum_{i} w_i x_{ij} \leq W_j, \forall j \in \{1, \ldots, \infty\}  \]
- \[  x_{ij} \in \{0,1\}, \forall i \in \{1, \ldots, N\}, j \in \{1, \ldots, \infty\}  \]


## MDP Formulation

**Actions:**
- \( a_i \in \{ bin_1, \ldots, bin_M \}, \forall i \in \{1, \ldots, N\} \): For every item \( i \), pack it into bin \( m \).

**States:**
- \( z_j \in \{0, 1, 2\}, \forall j \in \{1, \ldots, M\} \): For each bin \( j \), whether it is empty (0), non-empty (1), or capacity exceeded (2).
- (Not needed) \( s_i \in \{1, \ldots, M\}, \forall i \in \{1, \ldots, N\} \): Bin assigned to item \( i \).

**Transitions:**
- \( P(z_j \mid a_1, \ldots, a_N), \forall j \in \{1, \ldots, M\} \): For each bin \( j \),
  1. If no items are assigned to it, its state is empty: \( z_j = 0 \) if \( a_i \neq j, \forall i \in \{1, \ldots, N\} \),
  2. If the sum of weights of items assigned to it exceeds the capacity: \( z_j = 2 \) if \( \sum_{i} w_i a_i > W_j \),
  3. Bin is not empty but not capacity exceeded: \( z_j = 1 \).
- (Not needed) \( P(s_i \mid a_i), \forall i \in \{1, \ldots, N\} \): For every item \( i \), copy action over to state.

**Observations:**
- \( P(z_j \mid z_j), \forall j \in \{1, \ldots, M\} \): For every bin \( j \), copy weight constraint violation state over to observation.
- (Not needed) \( P(s_i \mid s_i), \forall i \in \{1, \ldots, N\} \): For every item \( i \), copy bin assignment state over to observation.

**Preferences:**
- (Not needed) \( P(s_i), \forall i \in \{1, \ldots, N\} \): Preference over every item \( i \) being assigned a bin is uniform: \( P = [0.5, 0.5] \).
- \( P(z_j), \forall j \in \{1, \ldots, M\} \): Preference over the state of each bin \( j \) as: empty > non-empty > capacity exceeded, e.g., [0.6, 0.4, 0]. This encourages the minimum number of bins used.


In [7]:
!git clone https://github.com/ran-wei-verses/pymdp.git


fatal: destination path 'pymdp' already exists and is not an empty directory.
/content/pymdp
Processing /content/pymdp
  Preparing metadata (setup.py) ... done
  Created wheel for inferactively-pymdp: filename=inferactively_pymdp-0.0.7.1-py3-none-any.whl size=59465 sha256=42c21c101eb8a71cbaeceac88ec5309f6428f849bdf1b0760586305edd1a8e88
  Stored in directory: /tmp/pip-ephem-wheel-cache-j8i5aqt4/wheels/6c/55/7e/ff6f739a81a005549be7e254ba199eccc1f7301546561e4a7b
Successfully built inferactively-pymdp
  Attempting uninstall: inferactively-pymdp
    Found existing installation: inferactively-pymdp 0.0.7.1
    Uninstalling inferactively-pymdp-0.0.7.1:
      Successfully uninstalled inferactively-pymdp-0.0.7.1


In [4]:
%cd pymdp
!pip install .
import sys
sys.path.append('/content/pymdp')  # Adjust based on the cloned path

/content/pymdp
Processing /content/pymdp
  Preparing metadata (setup.py) ... done
  Created wheel for inferactively-pymdp: filename=inferactively_pymdp-0.0.7.1-py3-none-any.whl size=59465 sha256=ecd9eb0a38b268e44c49aee023de5692a0f70a7733f9cc1006724e08f4362ce2
  Stored in directory: /tmp/pip-ephem-wheel-cache-oaogr8oq/wheels/6c/55/7e/ff6f739a81a005549be7e254ba199eccc1f7301546561e4a7b
Successfully built inferactively-pymdp
  Attempting uninstall: inferactively-pymdp
    Found existing installation: inferactively-pymdp 0.0.7.1
    Uninstalling inferactively-pymdp-0.0.7.1:
      Successfully uninstalled inferactively-pymdp-0.0.7.1


In [1]:
# Import necessary libraries
import numpy as np
import pymdp

# Import classes and functions from pymdp
from pymdp.agent import Agent
from pymdp import utils

# Pretty print for visual clarity
from pprint import pprint


In [2]:
np.random.seed(0)
# init dims
num_items = 5
num_bins = 10

item_weights = np.random.uniform(0, 10, size=(num_items,))
bin_capacity = np.random.uniform(5, 10, size=(num_bins,)) # no bins can fit more than 3 items of max weight

act_dim = num_bins
state_dim = 3 # choices = [empty, non-empty, exceeded]

# pymdp dims
act_dims = [act_dim for _ in range(num_items)]
state_dims = [state_dim for _ in range(num_bins)]

B_factor_list = [[] for _ in range(num_bins)]
B_factor_control_list = [list(range(num_items)) for _ in range(num_bins)]

print("\nact_dims")
pprint(act_dims)

print("\nstate_dims")
pprint(state_dims)

print("\nstate_dependencies")
pprint(B_factor_list)

print("\naction_dependencies")
pprint(B_factor_control_list)


act_dims
[10, 10, 10, 10, 10]

state_dims
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

state_dependencies
[[], [], [], [], [], [], [], [], [], []]

action_dependencies
[[0, 1, 2, 3, 4],
 [0, 1, 2, 3, 4],
 [0, 1, 2, 3, 4],
 [0, 1, 2, 3, 4],
 [0, 1, 2, 3, 4],
 [0, 1, 2, 3, 4],
 [0, 1, 2, 3, 4],
 [0, 1, 2, 3, 4],
 [0, 1, 2, 3, 4],
 [0, 1, 2, 3, 4]]


In [3]:
# transition matrix
B_template = pymdp.utils.random_B_matrix(
    state_dims,
    act_dims,
    B_factor_list=B_factor_list,
    B_factor_control_list=B_factor_control_list
)
B_array = np.array([np.zeros_like(a) for a in B_template], dtype=object)

print("transition dims")
pprint([a.shape for a in B_array])

TypeError: random_B_matrix() got an unexpected keyword argument 'B_factor_list'

In [ ]:
def create_constraint_factor(B, bin_index, act_dim, item_weights, bin_capacity):
    tensor = np.zeros_like(B)
    for a_1 in range(act_dim):
        for a_2 in range(act_dim):
            for a_3 in range(act_dim):
                for a_4 in range(act_dim):
                    for a_5 in range(act_dim):
                        assignment = np.array([a_1, a_2, a_3, a_4, a_5]) == bin_index
                        total_weight = np.sum(assignment * item_weights)

                        if not np.any(np.array([a_1, a_2, a_3, a_4, a_5]) == bin_index):
                            # print("bin empty")
                            tensor[0, a_1, a_2, a_3, a_4, a_5] = 1
                        else:
                            if total_weight > bin_capacity[bin_index]:
                                # print("exceeded")
                                tensor[2, a_1, a_2, a_3, a_4, a_5] = 1
                            else:
                                # print("not empty")
                                tensor[1, a_1, a_2, a_3, a_4, a_5] = 1
                        # return
    return tensor

In [ ]:
for bin_index in range(num_bins):
    B_array[bin_index] = create_constraint_factor(
        B_array[bin_index], bin_index, act_dim, item_weights, bin_capacity
    )

print("is normalized")
pprint([np.all(a.sum(0) == 1) for a in B_array])

In [ ]:
# assign observations
obs_dims = state_dims
A_factor_list = [[i] for i in range(len(state_dims))]

A_template = utils.random_A_matrix(
    obs_dims,
    state_dims,
    A_factor_list=A_factor_list
)
A_array = np.array([np.zeros_like(a) for a in A_template], dtype=object)

for i in range(len(A_array)):
    A_array[i] = np.eye(len(A_array[i]))

print("normalized")
print([a.shape for a in A_array])
print([a.sum(0).all() for a in A_array])

In [ ]:
# assign preference
C_template = utils.obj_array_uniform(
    obs_dims
)
C_array = np.array([np.zeros_like(a) for a in C_template], dtype=object)

# # assign constraint preference
for i in range(num_bins):
    C_array[i] = np.array([0.6, 0.4, 0.])

print("normalized")
print([a.shape for a in C_array])
print([a.sum(0) == 1 for a in C_array])

In [ ]:
agent = Agent(
    A=A_array,
    B=B_array,
    C=C_array,
    num_controls=act_dims,
    policy_len=1,
    A_factor_list=A_factor_list,
    B_factor_list=B_factor_list,
    B_factor_control_list=B_factor_control_list,
)

In [ ]:
q_pi, _ = agent.infer_policies()